In [2]:
%cd drive/MyDrive/SpectrometerLearner2/Zarya/
!ls

/content/drive/MyDrive/SpectrometerLearner2/Zarya
csv_processor.py			      __pycache__
hyperparameter_tuning_results.csv	      test.ipynb
hyperparameter_tuning_results_Sequential.csv  transformer.ipynb
MLPClassifier.ipynb


In [ ]:
from csv_processor import CSVProcessor
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

In [ ]:
processor = CSVProcessor("../Training Data")
X_train, y_train = processor.split()
processor = CSVProcessor("../Testing Data")
X_test, y_test = processor.split()

In [ ]:
scaler = StandardScaler()

# Fit the scaler on the training features and transform them
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test features using the fitted scaler
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Split the Data into Training and Test Sets
X_train, X_test = X_train_scaled, X_test_scaled

In [ ]:
param_grid = {
    'hidden_layer_sizes': [(64,), (128,), (256,), (64, 64), (128, 128), (256, 256)],
    'activation': ['relu', 'tanh'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate_init': [0.001, 0.01, 0.1],
}

# Create the model
model = MLPClassifier(random_state=42)

# Perform GridSearchCV with cross-validation
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Get the best hyperparameters and the corresponding best model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Save the results in a CSV file
results_df = pd.DataFrame(grid_search.cv_results_)
results_df.to_csv('hyperparameter_tuning_results.csv', index=False)

# Use the best model for predictions
y_train_pred = best_model.predict(X_train)

# Calculate accuracy on the training set
train_accuracy = accuracy_score(y_train, y_train_pred)
print("Training Accuracy:", train_accuracy)

In [ ]:
# Use the best model for testing (assuming you have X_test and y_test)
y_test_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)